In [ ]:

# Install the necessary libraries for the project
!pip install apache-beam scikit-learn joblib boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 341.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.5 MB/s eta 0:00:00

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
!unzip Bike-Sharing-Dataset.zip

--2024-10-09 18:38:51--  https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘Bike-Sharing-Dataset.zip’

Bike-Sharing-Datase     [  <=>               ] 273.43K  1.08MB/s    in 0.2s    

2024-10-09 18:38:51 (1.08 MB/s) - ‘Bike-Sharing-Dataset.zip’ saved [279992]

Archive:  Bike-Sharing-Dataset.zip
  inflating: Readme.txt              
  inflating: day.csv                 
  inflating: hour.csv                


In [ ]:
import pandas as pd
import boto3

df = pd.read_csv('day.csv')

df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [ ]:
from google.colab import userdata

your_access_key = userdata.get('AWS_ACCESS_KEY_ID')
your_secret_key = userdata.get('AWS_SECRET_ACCESS_KEY')
your_region = userdata.get('AWS_DEFAULT_REGION')

In [ ]:
s3 = boto3.client(
    's3',
    aws_access_key_id=your_access_key,
    aws_secret_access_key=your_secret_key,
    region_name=your_region
)

In [ ]:
df_clean = df[['temp', 'atemp', 'hum', 'windspeed', 'cnt']]

df_clean.to_csv('cleaned_bike_data.csv', index=False)

df_clean.head()

,temp,atemp,hum,windspeed,cnt
0,0.344167,0.363625,0.805833,0.160446,985
1,0.363478,0.353739,0.696087,0.248539,801
2,0.196364,0.189405,0.437273,0.248309,1349
3,0.200000,0.212122,0.590435,0.160296,1562
4,0.226957,0.229270,0.436957,0.186900,1600


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

df = pd.read_csv('day.csv')

X = df[['temp', 'atemp', 'hum', 'windspeed']]
y = df['cnt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

joblib.dump(model, 'bike_model.pkl')

print("Model trained and saved as 'bike_model.pkl'")

Model trained and saved as 'bike_model.pkl'


In [ ]:
import joblib

model = joblib.load('bike_model.pkl')

print(type(model))

sample_input = [[0.5, 0.6, 0.7, 0.2]]
prediction = model.predict(sample_input)
print(f"Prediction: {prediction}")

<class 'sklearn.linear_model._base.LinearRegression'>
Prediction: [4867.72718406]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.ml.inference import RunInference
from apache_beam.ml.inference.sklearn_inference import SklearnModelHandlerNumpy
import numpy as np
import boto3
import csv
from apache_beam.ml.inference.sklearn_inference import SklearnModelHandlerNumpy, ModelFileType

model_handler = SklearnModelHandlerNumpy(
    model_uri='bike_model.pkl',
    model_file_type=ModelFileType.JOBLIB
)

class PreprocessDataTransform(beam.PTransform):
    def expand(self, pcoll):
        return (
            pcoll
            | 'Preprocess Data' >> beam.ParDo(ConvertToNumpyArray())
        )

class ConvertToNumpyArray(beam.DoFn):
    def process(self, element):
        reader = csv.DictReader([element], fieldnames=['temp', 'atemp', 'hum', 'windspeed', 'cnt'])
        for row in reader:
            features = [
                float(row['temp']),
                float(row['atemp']),
                float(row['hum']),
                float(row['windspeed'])
            ]
            yield np.array(features)

In [ ]:
import boto3
import apache_beam as beam

def upload_to_s3(local_file, bucket, s3_file):
    try:
        s3.upload_file(local_file, bucket, s3_file)
        print(f"File {local_file} uploaded to S3 bucket {bucket} as {s3_file}")
    except Exception as e:
        print(f"Error uploading file: {e}")

class UploadPredictionsToS3(beam.DoFn):
    def __init__(self, bucket_name, file_name):
        self.bucket_name = bucket_name
        self.file_name = file_name
        self.prediction_buffer = []

    def process(self, element):
        self.prediction_buffer.append(str(element))

    def finish_bundle(self):
        local_file = 'predictions.txt'
        with open(local_file, 'w') as f:
            for prediction in self.prediction_buffer:
                f.write(prediction + '\n')
        upload_to_s3(local_file, self.bucket_name, self.file_name)
        self.prediction_buffer = []



def run_complete_pipeline():
    s3_bucket = 'hwapachebeamdm'
    s3_file = 'predicted_bike_data.txt'

    pipeline_options = PipelineOptions(runner='DirectRunner', streaming=False)

    with beam.Pipeline(options=pipeline_options) as p:

        raw_data = p | 'Read from CSV' >> beam.io.ReadFromText('cleaned_bike_data.csv', skip_header_lines=1)

        preprocessed_data = raw_data | 'Preprocess Data Transform' >> PreprocessDataTransform()

        predicted_data = preprocessed_data | 'Run Model Inference' >> RunInference(model_handler)

        predicted_data | 'Upload Predictions to S3' >> beam.ParDo(UploadPredictionsToS3('hwapachebeamdm', 'predicted_bike_data.txt'))

run_complete_pipeline()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature n

File predictions.txt uploaded to S3 bucket hwapachebeamdm as predicted_bike_data.txt
